In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

####################################################################################################
### Retrieves account information (NAV, positions, etc) from IB
### And saves it into csv files because we don't know how to export to DB for now
####################################################################################################
script_name = "IB_Account_Data"
max_time_hours = 1 / 60

In [2]:
####################################################################################################
### Imports
####################################################################################################
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
import init
import utils as ut
import execution_utils as xu
import db
import pandas as pd

####################################################################################################
### Main initialization - Start                                              2023-08-28 07:19:47 ###
####################################################################################################
### Script Name: execution_utils                                             2023-08-28 07:19:47 ###
### Script PID: 3118684                                                      2023-08-28 07:19:47 ###
### Initializating application parameters                                    2023-08-28 07:19:47 ###
### This computer: H                                                         2023-08-28 07:19:47 ###
### System date: 2023-08-28                                                  2023-08-28 07:19:47 ###
### Current directory: /home/fls/Models/Ventura/HD/Scripts/Python            2023-08-28 07:19:47 ###


In [49]:
####################################################################################################
### Script initialization
####################################################################################################
init.scriptInit(script_name, max_time_hours)

####################################################################################################
### Main initialization - Start                                              2023-08-28 08:07:54 ###
####################################################################################################
### Script Name: IB_Account_Data                                             2023-08-28 08:07:54 ###
### Script PID: 3118684                                                      2023-08-28 08:07:54 ###
### Initializating application parameters                                    2023-08-28 08:07:54 ###
### This computer: H                                                         2023-08-28 08:07:54 ###
### System date: 2023-08-28                                                  2023-08-28 08:07:54 ###
### Current directory: /home/fls/Models/Ventura/HD/Scripts/Python            2023-08-28 08:07:54 ###
### Loading main data tables                                                 2023-08-28 08:

True

In [50]:
####################################################################################################
### Script variables
####################################################################################################
account_id_list = [1,2]
file_path = init.DIRECTORY_DATA + "Account_Data/"
file_path_account = file_path + "Account/"
file_path_px_position = file_path + "Px_Position/"
columns_px_position = ["account_id", "conid", "symbol", "secType", "primaryExchange", 
    "currency", "price", "position"]

In [112]:
####################################################################################################
### Classes
####################################################################################################
class IBAppBook(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.account_data = pd.DataFrame()
        self.px_position = pd.DataFrame()

    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson):
        print("IB Error ", reqId, " ", errorCode, " ", errorString)
        if errorCode == 502:
            ut.printBanner("Error - Not connected - exiting", False)
            self.stop()
    
    def nextValidId(self, orderId):
        print("Start")
        self.start()     

    def addPxPositionRow(self, contract, price, position):
     #   print("addPxPositionRow")
        try:
            dat = pd.DataFrame({
                "contract":[contract],
                "price":[price],
                "position":[position]
                })
            self.px_position = pd.concat([self.px_position, dat])
        except:
            pass
        
    def addAccountDataRow(self, key, value, ccy):
    #    print("addAccountDataRow")
    #    print([key, value, ccy])
        dat = pd.DataFrame({
            "key":[key],
            "value":[value],
            "ccy":[ccy]
            })
        self.account_data = pd.concat([self.account_data, dat])
        
    def updatePortfolio(self, contract: Contract, position: float, marketPrice: float, marketValue: float,
                        averageCost: float, unrealizedPNL: float, realizedPNL: float, accountName: str):
        print("updatePortfolio")
        print([contract, position, marketPrice, marketValue, 
            averageCost, unrealizedPNL, realizedPNL, accountName])
        self.addPxPositionRow(contract, marketPrice, position) 
        
    def updateAccountValue(self, key, val, currency, accountName):
    #    print("updateAccountValue")
    #    print([key, val, currency])
        self.addAccountDataRow(key, val, currency)
    
    def accountDownloadEnd(self, account):
        print("accountDownloadEnd")
        self.finalFormatting()
        self.stop()
        
    def finalFormatting(self):
        print("finalFormatting")
        print(self.px_position)
        if len(self.px_position) > 0:
            self.px_position = formatPxPositionTable(self.px_position, self.account_id)

    def start(self):
        print("start")
        self.reqAccountUpdates(True, "")
        
    def stop(self):
        print("stop")
        self.reqAccountUpdates(False, "")
        self.done = True
        self.disconnect()

In [84]:
####################################################################################################
### Sub routines
####################################################################################################
def correctFTSEPrice(dat):
    # TO BE CORRECTED WITH A 'MULTIPLIER' PARAMETER IN static_future_contract
    pos_ftse = (dat["symbol"] == "Z")
    dat.price[pos_ftse] = 100 * dat.price[pos_ftse]
    return dat

def formatPxPositionTable(dat, account_id):
    print(dat)
    dat = dat \
        .assign(
            account_id = account_id,
            conid = dat.contract.map(lambda x: x.conId),
            symbol = dat.contract.map(lambda x: x.symbol),
            secType = dat.contract.map(lambda x: x.secType),
            primaryExchange = dat.contract.map(lambda x: x.primaryExchange),
            currency = dat.contract.map(lambda x: x.currency)
            ) \
        [columns_px_position]
    dat = correctFTSEPrice(dat)
    return dat

@ut.tryDiagnosticDat()
def saveAccountFile(dat, account_id):
    file_name = f"account_{account_id}_{init.char_start_time}.csv"
    dat.to_csv(file_path_account + file_name, index=False)
    file_name = f"account_{account_id}_last.csv"
    dat.to_csv(file_path_account + file_name, index=False)
    return dat

@ut.tryDiagnosticDat()    
def savePxPositionFile(dat, account_id):
    file_name = f"px_position_{account_id}_{init.char_start_time}.csv"
    dat.to_csv(file_path_px_position + file_name, index=False)
    file_name = f"px_position_{account_id}_last.csv"
    dat.to_csv(file_path_px_position + file_name, index=False)
    return dat

def tryToFindQuoteInUSD(dat_px_position, ccy_account):
    return dat_px_position[
        (dat_px_position["secType"] == "CASH")
        & (dat_px_position["symbol"] == ccy_account)
        & (dat_px_position["currency"] == "USD")
    ]

def tryToConvertFXRateToUSDUsingAccount(dat_account, ccy_account):
    fx_account = float(dat_account[
        (dat_account["key"] == "ExchangeRate")
        & (dat_account["ccy"] == ccy_account)
    ]["value"].values[0])
    fx_usd = float(dat_account[
        (dat_account["key"] == "ExchangeRate")
        & (dat_account["ccy"] == "USD")
    ]["value"].values[0])
    return fx_account / fx_usd
    

def findUSDQuotedInCCY(dat_px_position, ccy_account):
    dat_fx = dat_px_position[
        (dat_px_position["secType"] == "CASH")
        & (dat_px_position["symbol"] == "USD")
        & (dat_px_position["currency"] == ccy_account)
    ]
    fx = 1 / float(dat_fx["price"].values[0])
    return fx

def findFXinPxPosition(dat_px_position, account_id):
    ACCOUNTS = db.loadTableLocal("account")
    ccy_id_account = ACCOUNTS["ccy_id"][ACCOUNTS["account_id"] == account_id].values[0]

    ccy_account = init.CURRENCIES["ccy"][init.CURRENCIES["ccy_id"] == ccy_id_account].values[0]

    dat_fx = tryToFindQuoteInUSD(dat_px_position, ccy_account)
    if len(dat_fx) > 0:
        fx = float(dat_fx["price"].values[0])
    else:
        fx = findUSDQuotedInCCY(dat_px_position, ccy_account)
    return fx

def findFXinAccount(dat_account, account_id):
    ACCOUNTS = db.loadTableLocal("account")
    ccy_id_account = ACCOUNTS["ccy_id"][ACCOUNTS["account_id"] == account_id].values[0]
    ccy_account = init.CURRENCIES["ccy"][init.CURRENCIES["ccy_id"] == ccy_id_account].values[0]
    fx = tryToConvertFXRateToUSDUsingAccount(dat_account, ccy_account)
    return fx

In [85]:
@ut.tryDiagnosticDat()
def saveNAVToDB(dat_account, dat_px_position, account_id):
    nav = float(dat_account["value"][dat_account["key"] == "NetLiquidation"].values[0])
    fx = findFXinAccount(dat_account, account_id)
    nav_usd = nav * fx

    sql_q = """INSERT INTO book_nav
        (account_id, date, timestamp, nav_ccy, nav_usd)
        VALUES ({}, '{}', '{}', {}, {})""".format(
            account_id,
            init.TODAY_STR,
            init.char_start_time,
            nav,
            nav_usd
        )
    db.executeSQL(sql_q)

    sql_q = """SELECT * FROM book_nav 
        WHERE account_id = {} AND date = '{}' AND timestamp = '{}'""".format(
            account_id,
            init.TODAY_STR,
            init.char_start_time
        )
    return db.select(sql_q)

def extractAccountFXData(dat_i, dat_fx):
    dat_fx_i = dat_i[
        (dat_i["key"] == "ExchangeRate") 
            & (dat_i["ccy"] != "BASE")
         ] \
        .assign(value = lambda x: float(x.value))
    fx_usd = dat_fx_i[dat_fx_i["ccy"] == "USD"]["value"].values[0]
    dat_fx_i["value"] = dat_fx_i["value"].apply(lambda x: x / fx_usd)
    return pd.concat([dat_fx, dat_fx_i])

def extractAccountPositionData(dat_i, dat_pos, account_id):
    dat_pos_i = dat_i[
        (dat_i["key"] == "TotalCashBalance")
            & (dat_i["ccy"] != "BASE")
        ] \
        .assign(
            account_id = account_id,
            value = lambda x: float(x.value)
            )
    
    return pd.concat([dat_pos, dat_pos_i])

def formatFXPrices(dat_fx):
    return dat_fx \
        .drop("key", axis=1) \
        .rename({"ccy":"code", "value":"price"}, axis=1) \
        .groupby("code", as_index=False).agg({"price":"mean"}) \
        .merge(init.ASSETS, on="code", how="left") \
        .assign(timestamp = init.char_start_time) \
        [["asset_id", "timestamp", "price"]]

def formatPositions(dat_pos, dat_fx): 
    return dat_pos \
       .drop("key", axis=1) \
       .rename({"ccy":"code", "value":"position"}, axis=1) \
       .merge(init.ASSETS, on="code", how="left") \
       .assign(
           timestamp = init.char_start_time,
           position = lambda x: float(x.position)
           ) \
       [["account_id", "asset_id", "timestamp", "position"]]
    
@ut.tryDiagnosticNone()
def saveFXPxPositionIntoDB():
    dat_fx = pd.DataFrame()
    dat_pos = pd.DataFrame()
    for account_id in account_id_list:
        file_name = "account_{}_{}.csv".format(account_id, init.char_start_time)
        dat_i = pd.read_csv(file_path_account + file_name)        
        dat_fx = extractAccountFXData(dat_i, dat_fx)
        dat_pos = extractAccountPositionData(dat_i, dat_pos, account_id)

    dat_fx = formatFXPrices(dat_fx)
    db.appendToTable("live_px_fx_book", dat_fx)
        
    dat_pos = formatPositions(dat_pos, dat_fx)
    db.appendToTable("book_live_position_fx", dat_pos)
    return [dat_fx, dat_pos]

def extractAccountFutureData(dat_i, dat_fut):
    # dat_fut_i = dat_i[dat_i["secType"] == "FUT"]
    # dat_fut_i["account_id"] = dat_fut_i["account_id"].map(int)
    # dat_fut_i["conid"] = dat_fut_i["conid"].map(int)
    # dat_fut_i["price"] = dat_fut_i["price"].map(float)
    # dat_fut_i["position"] = dat_fut_i["position"].map(float)
    # dat_fut_i = dat_fut_i[["account_id", "conid", "price", "position"]]
    # return dat_fut_i
    
    return dat_i \
        [dat_i["secType"] == "FUT"] \
        .assign(
            account_id = dat_i.account_id.map(int),
            conid = dat_i.conid.map(int),
            price = dat_i.price.map(float),
            position = dat_i.position.map(float)
            ) \
        [["account_id", "conid", "price", "position"]]

def formatFuturePrices(dat):
    dat_px = dat.groupby("conid", as_index=False).agg({"price":"mean"})
    dat_px["timestamp"] = init.char_start_time
    dat_px = dat_px[["conid", "timestamp", "price"]]
    return dat_px

def formatFuturePositions(dat):
    dat["timestamp"] = init.char_start_time
    dat = dat[["account_id", "conid", "timestamp", "position"]]
    return dat

@ut.tryDiagnosticNone()
def saveFuturePxPositionIntoDB():
    dat_fut = pd.DataFrame()
    for account_id in account_id_list:
        file_name = "px_position_{}_{}.csv".format(account_id, init.char_start_time)
        dat_i = pd.read_csv(file_path_px_position + file_name)        
        dat_fut = extractAccountFXData(dat_i, dat_fut)
    
    dat_px = formatFuturePrices(dat_fut)
    db.appendToTable("live_px_future", dat_px)
        
    dat_pos = formatFuturePositions(dat_fut)
    db.appendToTable("book_live_position_future", dat_pos)
    return [dat_px, dat_pos]

@ut.tryDiagnosticNone()
def getBookLivePriceAndPositions(account_id):
    ut.printBanner("Retrieving data for account " + str(account_id))
    ib_app = IBAppBook()
    ib_port = xu.getIBPort(account_id)
    ib_client_id = xu.getIBClientId(script_name)
    ib_app.connect("127.0.0.1", ib_port, ib_client_id)
    ib_app.account_id = account_id
    ib_app.run()    
    global AAA
    AAA = ib_app.px_position
    saveAccountFile(ib_app.account_data, account_id)
    savePxPositionFile(ib_app.px_position, account_id)
    saveNAVToDB(ib_app.account_data, ib_app.px_position, account_id)

In [54]:
####################################################################################################
### Script
####################################################################################################
if __name__ == "__main__":
    for account_id in account_id_list:
        getBookLivePriceAndPositions(account_id)
    saveFXPxPositionIntoDB()
#    saveFuturePxPositionIntoDB()
    init.scriptFinish(script_name)

####################################################################################################
### getBookLivePriceAndPositions - Starting                                  2023-08-28 08:09:36 ###
####################################################################################################
### Retrieving data for account 1                                            2023-08-28 08:09:36 ###
####################################################################################################
IB Error  -1   2104   Market data farm connection is OK:cafarm
IB Error  -1   2104   Market data farm connection is OK:hfarm
IB Error  -1   2104   Market data farm connection is OK:eufarmnj
IB Error  -1   2104   Market data farm connection is OK:cashfarm
IB Error  -1   2104   Market data farm connection is OK:usfuture
IB Error  -1   2104   Market data farm connection is OK:usfuture.nj
IB Error  -1   2104   Market data farm connection is OK:usfarm.nj
IB Error  -1   2104   Market data farm co

IB Error  -1   2104   Market data farm connection is OK:usfarm
IB Error  -1   2106   HMDS data farm connection is OK:cashhmds
IB Error  -1   2106   HMDS data farm connection is OK:fundfarm
IB Error  -1   2106   HMDS data farm connection is OK:ushmds
IB Error  -1   2158   Sec-def data farm connection is OK:secdefnj
['AccountCode', 'U7336562', '']
['AccountCode', 'U7336562', '']
['AccountOrGroup', 'U7336562', 'AUD']
['AccountOrGroup', 'U7336562', 'AUD']
['AccountOrGroup', 'U7336562', 'BASE']
['AccountOrGroup', 'U7336562', 'BASE']
['AccountOrGroup', 'U7336562', 'EUR']
['AccountOrGroup', 'U7336562', 'EUR']
['AccountOrGroup', 'U7336562', 'GBP']
['AccountOrGroup', 'U7336562', 'GBP']
['AccountOrGroup', 'U7336562', 'HKD']
['AccountOrGroup', 'U7336562', 'HKD']
['AccountOrGroup', 'U7336562', 'ILS']
['AccountOrGroup', 'U7336562', 'ILS']
['AccountOrGroup', 'U7336562', 'JPY']
['AccountOrGroup', 'U7336562', 'JPY']
['AccountOrGroup', 'U7336562', 'KRW']
['AccountOrGroup', 'U7336562', 'KRW']
['AccountO

### saveFXPxPositionIntoDB - Error while running:                            2023-08-28 08:09:36 ###
[Errno 2] No such file or directory: '/home/fls/Data/Ventura/HD/Account_Data/Account/account_1_2023-08-28 07:19:47.csv'
### saveFXPxPositionIntoDB - Done. Result:                                   2023-08-28 08:09:36 ###
None
####################################################################################################
####################################################################################################
### Script Complete - IB_Account_Data                                        2023-08-28 08:09:36 ###
####################################################################################################


In [59]:
account_id=2

In [62]:
getBookLivePriceAndPositions(2)

####################################################################################################
### getBookLivePriceAndPositions - Starting                                  2023-08-28 08:11:35 ###
####################################################################################################
### Retrieving data for account 2                                            2023-08-28 08:11:35 ###
####################################################################################################
IB Error  -1   2104   Market data farm connection is OK:usfarm
IB Error  -1   2106   HMDS data farm connection is OK:cashhmds
IB Error  -1   2106   HMDS data farm connection is OK:fundfarm
IB Error  -1   2106   HMDS data farm connection is OK:ushmds
IB Error  -1   2158   Sec-def data farm connection is OK:secdefnj
['AccountCode', 'U7336562', '']
['AccountCode', 'U7336562', '']
['AccountOrGroup', 'U7336562', 'AUD']
['AccountOrGroup', 'U7336562', 'AUD']
['AccountOrGroup', 'U7336562', 'BASE']


['RealizedPnL', '0.00', 'BASE']
['RealizedPnL', '0.00', 'BASE']
['RealizedPnL', '0.00', 'EUR']
['RealizedPnL', '0.00', 'EUR']
['RealizedPnL', '0.00', 'GBP']
['RealizedPnL', '0.00', 'GBP']
['RealizedPnL', '0.00', 'HKD']
['RealizedPnL', '0.00', 'HKD']
['RealizedPnL', '0.00', 'ILS']
['RealizedPnL', '0.00', 'ILS']
['RealizedPnL', '0.00', 'JPY']
['RealizedPnL', '0.00', 'JPY']
['RealizedPnL', '0.00', 'KRW']
['RealizedPnL', '0.00', 'KRW']
['RealizedPnL', '0.00', 'NZD']
['RealizedPnL', '0.00', 'NZD']
['RealizedPnL', '0.00', 'SGD']
['RealizedPnL', '0.00', 'SGD']
['RealizedPnL', '0.00', 'TRY']
['RealizedPnL', '0.00', 'TRY']
['RealizedPnL', '0.00', 'USD']
['RealizedPnL', '0.00', 'USD']
['RealizedPnL', '0.00', 'ZAR']
['RealizedPnL', '0.00', 'ZAR']
['SegmentTitle-S', 'Securities', '']
['SegmentTitle-S', 'Securities', '']
['StockMarketValue', '0.00', 'AUD']
['StockMarketValue', '0.00', 'AUD']
['StockMarketValue', '0.00', 'BASE']
['StockMarketValue', '0.00', 'BASE']
['StockMarketValue', '0.00', 'EUR'

In [113]:
ib_app = IBAppBook()
ib_port = xu.getIBPort(account_id)

In [114]:
ib_port

7499

In [115]:
script_name

'IB_Account_Data'

In [116]:
ib_client_id = xu.getIBClientId(script_name)

In [117]:
ib_client_id

123

In [118]:
ib_app.connect("127.0.0.1", ib_port, ib_client_id)

In [119]:
ib_app.account_id = account_id

In [120]:
ib_app.run()

Start
start
IB Error  -1   2104   Market data farm connection is OK:usfarm
IB Error  -1   2106   HMDS data farm connection is OK:cashhmds
IB Error  -1   2106   HMDS data farm connection is OK:fundfarm
IB Error  -1   2106   HMDS data farm connection is OK:ushmds
IB Error  -1   2158   Sec-def data farm connection is OK:secdefnj
accountDownloadEnd
finalFormatting
Empty DataFrame
Columns: []
Index: []
stop


In [47]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from threading import Timer

class TestApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)

    def error(self, reqId, errorCode, errorString):
        print("Error: ", reqId, " ", errorCode, " ", errorString)

    def nextValidId(self, orderId):
        self.start()

    def updatePortfolio(self, contract: Contract, position: float, marketPrice: float, marketValue: float,
                        averageCost: float, unrealizedPNL: float, realizedPNL: float, accountName: str):
        print("UpdatePortfolio.", "Symbol:", contract.symbol, "SecType:", contract.secType, "Exchange:", contract.exchange,
              "Position:", position, "MarketPrice:", marketPrice, "MarketValue:", marketValue, "AverageCost:", averageCost,
              "UnrealizedPNL:", unrealizedPNL, "RealizedPNL:", realizedPNL, "AccountName:", accountName)

    def updateAccountValue(self, key: str, val: str, currency: str, accountName: str):
        print("UpdateAccountValue. Key:", key, "Value:", val, "Currency:", currency, "AccountName:", accountName)

    def updateAccountTime(self, timeStamp: str):
        print("UpdateAccountTime. Time:", timeStamp)

    def accountDownloadEnd(self, accountName: str):
        print("AccountDownloadEnd. Account:", accountName)

    def start(self):
        # Account number can be omitted when using reqAccountUpdates with single account structure
        self.reqAccountUpdates(True, "")

    def stop(self):
        self.reqAccountUpdates(False, "")
        self.done = True
        self.disconnect()

def main():
    app = TestApp()
    app.connect("127.0.0.1", 7499, 0)

    Timer(5, app.stop).start()
    app.run()

if __name__ == "__main__":
    main()

TypeError: error() takes 4 positional arguments but 5 were given